### Example of Reshaping Data  from wide to long format

### Environment set up and data

In [2]:
# Import the relevant libraries...
# By the way, I am ussing Python 3
import pandas as pd
# Load the raw data using the ExcelFile object
data = pd.ExcelFile('reshaping_data.xlsx')
data

In [8]:
data.parse(sheetname='ABC_inc', skiprows=7).head(10)
tabnames = data.sheet_names
tabnames

['ABC_inc', 'HIJ_inc', 'OPQ_inc', 'XYZ_inc']

In [10]:
# Lets skip the first 7 rows and look at our data starting row 8
i = 0 
df = data.parse(sheet_name=tabnames[i], skiprows=7)
df.head()


,Unnamed: 0,district,province,partner,funding_source,Unnamed: 5,Unnamed: 6,2017,2017.1,2017.2,...,Unnamed: 21,2020,2020.1,2020.2,2020.3,Unnamed: 26,2021,2021.1,2021.2,2021.3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
1,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409
2,NaN,District 2,Region 3,partner 6,Souce 5,NaN,NaN,722,232,1848,...,NaN,810,664,452,3665,NaN,989,374,1790,4320
3,NaN,District 3,Region 1,partner 1,Souce 2,NaN,NaN,545,585,1736,...,NaN,1890,736,1414,5311,NaN,1215,112,1475,2824
4,NaN,District 4,Region 3,partner 6,Souce 5,NaN,NaN,631,1413,31,...,NaN,1646,960,209,2956,NaN,1392,936,701,4903


In [11]:
# Standarize existing columns and create a new ones

In [14]:
# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x)[:4] for x in cols1]
cols1

['Unna',
 'dist',
 'prov',
 'part',
 'fund',
 'Unna',
 'Unna',
 '2017',
 '2017',
 '2017',
 '2017',
 'Unna',
 '2018',
 '2018',
 '2018',
 '2018',
 'Unna',
 '2019',
 '2019',
 '2019',
 '2019',
 'Unna',
 '2020',
 '2020',
 '2020',
 '2020',
 'Unna',
 '2021',
 '2021',
 '2021',
 '2021']

In [16]:
# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x) for x in cols2]
cols2

['nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total',
 'nan',
 '10-14yrs',
 '15-29yrs',
 '30+yrs',
 'Total']

In [17]:
# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]
# Assign new column names to the dataframe
df.columns = cols
df.head(1)

,Unna_nan,dist_nan,prov_nan,part_nan,fund_nan,Unna_nan,Unna_nan,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,...,Unna_nan,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,Unna_nan,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total


In [18]:
# Drop empty columns, Rename the useful columns
# Note when you drop, you should specify axis=1 for columns and axis=0 for rows
df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns=    {'dist_nan':'district',                                              'prov_nan': 'province',                                                   'part_nan':'partner',                                                   'fund_nan':'financing_source'})
df.head(2)

,district,province,partner,financing_source,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,2017_Total,2018_10-14yrs,2018_15-29yrs,...,2019_30+yrs,2019_Total,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
1,District 1,Region 1,partner 1,Souce 2,1296,383,1571,3250,189,854,...,491,2256,1906,1925,931,5465,61,353,1091,2409
2,District 2,Region 3,partner 6,Souce 5,722,232,1848,2802,972,69,...,245,2957,810,664,452,3665,989,374,1790,4320


### Reusable Function

In [2]:
# Now define a function for parsing other funder targets data
def ReshapeFunc(excel_obj, i):
    """ Takes in an excel file object with multiple tabs in a wide format, and a specified index
    of the tab to be parsed and reshaped. Returns a dataframe of the specified tab
    reshaped to long format"""
    
    tabnames = data.sheet_names
    assert i < len(tabnames), "Your tab index exceeds the number of available tabs, try a lower number" 
    
    # parse and clean columns
    df = excel_obj.parse(sheetname=tabnames[i], skiprows=7)
    cols1 = [str(x)[:4] for x in list(df.columns)]
    cols2 = [str(x) for x in list(df.iloc[0,:])]
    cols = [x+"_"+y for x,y in zip(cols1,cols2)]
    df.columns = cols
    df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns={'dist_nan':'district',
                                                       'prov_nan': 'province',
                                                       'part_nan':'partner',
                                                       'fund_nan':'financing_source'})
    # new columns, drop some and change data type
    df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
    df.drop([c for c in df.columns if "Total" in c], axis=1, inplace= True) 
    
    for c in [c for c in df.columns if "yrs" in c]:
        df[c] = df[c].apply(lambda x: pd.to_numeric(x))

    # reshape - indexing, pivoting and stacking
    idx = ['district','province', 'partner','financing_source','main_organization']
    multi_indexed_df = df.set_index(idx)
    stacked_df = multi_indexed_df.stack(dropna=False)
    stacked_df.head(25) # check out the results!
    long_df = stacked_df.reset_index()
    
    # clean up and finalize
    col_str = long_df.level_5.str.split("_") 
    long_df['target_year'] = [x[0] for x in col_str] 
    long_df['target_age'] = [x[1] for x in col_str]
    long_df['target_quantity'] = long_df[0] # rename this column
    df_final = long_df.drop(['level_5', 0], axis=1)

    return df_final

### Use function, concatenate and save results

In [3]:
# Check that our function works:
check_df = ReshapeFunc(data, 2)
check_df.head(2)

,district,province,partner,financing_source,main_organization,target_year,target_age,target_quantity
0,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,10-14yrs,1
1,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,15-29yrs,974


In [4]:
# Apply the function iteratively, concatenate and save
dfs_list = [ReshapeFunc(data, i) for i in range(4)]
concat_dfs  = pd.concat(dfs_list)
concat_dfs.to_excel("reshaping_result_long_format.xlsx")